In [1]:
!pip install --upgrade transformers
!pip install simpletransformers

     |████████████████████████████████| 1.4MB 18.9MB/s 
     |████████████████████████████████| 2.9MB 49.1MB/s 
     |████████████████████████████████| 890kB 50.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=16af0099aca2ab8de6391bd3fd55b3548b635e0fd7487fd887184636fbe5bcc1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 204kB 10.7MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 1.1MB 31.7MB/s 
     |████████████████████████████████| 1.8MB 46.1MB/s 
     |████████████████████████████████| 7.4MB 49.9MB/s 
     |████████████████████████████████| 317kB 52.5MB/s 
     |████████████████████████████████| 102kB 13.0MB/s 
     |████████████████████████████████| 133kB 55.8MB/s 
     |████████████████████████████

In [2]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import numpy as np


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import zipfile
with zipfile.ZipFile('/content/Alldata.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

**For this rask I'll change the problem nature to a simple classification problem where we have 1 sentense and 1 label corresponding to that. Label is 0 if sentense make the sense and 1 if it doesn't**

**Reading Traning Data**

In [5]:
df_train_data = pd.read_csv("./Alldata/subtaskA_data_all.csv")
df_train_label = pd.read_csv("./Alldata/subtaskA_answers_all.csv", names = ["id", "label"])



train_data = df_train_data.loc[:,['sent0','sent1']].to_numpy()
train_label = df_train_label.loc[:,['label']].to_numpy()

In [6]:
df_train_data.head()

,id,sent0,sent1
0,0,He poured orange juice on his cereal.,He poured milk on his cereal.
1,1,He drinks apple.,He drinks milk.
2,2,Jeff ran a mile today,"Jeff ran 100,000 miles today"
3,3,A mosquito stings me,I sting a mosquito
4,4,A niece is a person.,A giraffe is a person.


**now we'll check is distribution of the two classes**

In [7]:
df_train_label['label'].value_counts(normalize = True)

1    0.5021
0    0.4979
Name: label, dtype: float64

In [8]:
df_trial_data = pd.read_csv("./Alldata/taskA_trial_data.csv")
df_trial_label = pd.read_csv("./Alldata/taskA_trial_answer.csv", names = ["id", "label"])


trial_data = df_trial_data.loc[:,['sent0','sent1']].to_numpy()
trial_label = df_trial_label.loc[:,['label']].to_numpy()

df_dev_data = pd.read_csv("./Alldata/subtaskA_dev_data.csv")
df_dev_label = pd.read_csv("./Alldata/subtaskA_gold_answers.csv", names = ["id", "label"])



dev_data = df_dev_data.loc[:,['sent0','sent1']].to_numpy()
dev_label = df_dev_label.loc[:,['label']].to_numpy()

**Reading Test data**

In [9]:
df_test_data = pd.read_csv("./Alldata/subtaskA_test_data.csv")
df_test_label = pd.read_csv("./Alldata/subtaskA_gold_test_answers.csv", names = ["id", "label"])




test_data = df_test_data.loc[:,['sent0','sent1']].to_numpy()
test_label = df_test_label.loc[:,['label']].to_numpy()

**Preparing data for traning**

**Now converting data for simple classification problem type**

In [10]:
train = []
test = []

for x in range(len(train_data)):
    t_sent0, t_sent1 = train_data[x]
    if train_label[x][0] == 0:
        train.append([t_sent0, 1])
        train.append([t_sent1, 0])
    else:
        train.append([t_sent0, 0])
        train.append([t_sent1, 1])   


for x in range(len(trial_data)):
    t_sent0, t_sent1 = trial_data[x]
    if trial_label[x][0] == 0:
        train.append([t_sent0, 1])
        train.append([t_sent1, 0])
    else:
        train.append([t_sent0, 0])
        train.append([t_sent1, 1])      


for x in range(len(dev_data)):
    t_sent0, t_sent1 = dev_data[x]
    if dev_label[x][0] == 0:
        train.append([t_sent0, 1])
        train.append([t_sent1, 0])
    else:
        train.append([t_sent0, 0])
        train.append([t_sent1, 1])           


for x in range(len(test_data)):
    t_sent0, t_sent1 = test_data[x]
    if test_label[x][0] == 0:
        test.append([t_sent0, 1])
        test.append([t_sent1, 0])
    else:
        test.append([t_sent0, 0])
        test.append([t_sent1, 1])    


In [11]:
train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)

In [12]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [45]:
model_args = ClassificationArgs(num_train_epochs=5, overwrite_output_dir=True, manual_seed=42, train_batch_size = 128, max_seq_length = 25,  save_model_every_epoch = False)
# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-large',args=model_args) # You can set class weights by using the optional weight argument


INFO:filelock:Lock 139670761687640 acquired on /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373.lock


INFO:filelock:Lock 139670761687640 released on /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373.lock
INFO:filelock:Lock 139668509460408 acquired on /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352.lock


INFO:filelock:Lock 139668509460408 released on /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352.lock


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.o

INFO:filelock:Lock 139668452021024 released on /root/.cache/huggingface/transformers/7c1ba2435b05451bc3b4da073c8dec9630b22024a65f6c41053caccf2880eb8f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab.lock
INFO:filelock:Lock 139670731669456 acquired on /root/.cache/huggingface/transformers/20b5a00a80e27ae9accbe25672aba42ad2d4d4cb2c4b9359b50ca8e34e107d6d.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


INFO:filelock:Lock 139670731669456 released on /root/.cache/huggingface/transformers/20b5a00a80e27ae9accbe25672aba42ad2d4d4cb2c4b9359b50ca8e34e107d6d.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


**Now traning the model**

In [46]:

# Train the model
import torch
torch.cuda.empty_cache()
model.train_model(train_df)



/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:387: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


(1020, 0.24907390199072074)

In [47]:
# # Train the model
# import torch
# torch.cuda.empty_cache()
# model.train_model(train_df)

**Now testing the acc against train dataset**

In [48]:
# Evaluate the model
from sklearn.metrics import accuracy_score


result, model_outputs, wrong_predictions = model.eval_model(train_df, acc=accuracy_score)
print("Train data acc : ", result)


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9845838961588153, 'tp': 12872, 'tn': 12963, 'fp': 55, 'fn': 146, 'acc': 0.992279920110616, 'eval_loss': 0.0253859512477694}



Train data acc :  {'mcc': 0.9845838961588153, 'tp': 12872, 'tn': 12963, 'fp': 55, 'fn': 146, 'acc': 0.992279920110616, 'eval_loss': 0.0253859512477694}


**Now testing the acc against test dataset**

In [49]:
# Evaluate the model
from sklearn.metrics import accuracy_score


result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=accuracy_score)
print("Test data acc : ", result)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.7738983794000517, 'tp': 851, 'tn': 921, 'fp': 79, 'fn': 149, 'acc': 0.886, 'eval_loss': 0.4824936160969082}



Test data acc :  {'mcc': 0.7738983794000517, 'tp': 851, 'tn': 921, 'fp': 79, 'fn': 149, 'acc': 0.886, 'eval_loss': 0.4824936160969082}


**Now working on preparing prediction w.r.t out orignal problem
where we have 2 sentenses and 1 label corresponding to that which represent which one isn't making sense**

In [50]:
pred = []
for x in model_outputs:
    pred.append(np.argmax(x))


In [51]:
test_id = df_test_label.loc[:,['id']].to_numpy()

In [69]:

import random
preds_list = []
x = 0
y = 0
while x < (len(pred)-1):
    if pred[x]== pred[x+1] and (pred[x] == 0 or pred[x] == 1):
        
        preds_list.append([test_id[y][0], 1])
        
    elif pred[x] == 1:
        
        preds_list.append([test_id[y][0], 0])
        
    elif pred[x+1] == 1:
        
        preds_list.append([test_id[y][0], 1])
    
    x += 2
    y += 1

    

**Now calculating and displaying accuracy on orignal test set form**


In [70]:
acc = 0.0
for x in range(len(test_label)):
    if test_label[x][0] == preds_list[x][1]:
        acc += 1

print("Acc : ", acc/len(test_label))

Acc :  0.896


**Now writing predictions to file for submission**

In [71]:
f = open("subtaskA_answers.csv", "w")
for row in preds_list:
    
    f.write(str(row[0]) +","+str(row[1]) + "\n")
    
f.close()